In [ ]:
import httplib2
from apiclient.discovery import build
from oauth2client.client import flow_from_clientsecrets
from oauth2client.file import Storage
from oauth2client import tools
import argparse
 
CLIENT_SECRETS = 'client_secrets.json'
 
# The Flow object to be used if we need to authenticate.
FLOW = flow_from_clientsecrets(
    CLIENT_SECRETS,
    scope='https://www.googleapis.com/auth/analytics.readonly',
    message='%s is missing' % CLIENT_SECRETS
    )
 
# A file to store the access token
TOKEN_FILE_NAME = 'credentials.dat'
 
def prepare_credentials():
    parser = argparse.ArgumentParser(parents=[tools.argparser])
    flags = parser.parse_args()
    # Retrieve existing credendials
    storage = Storage(TOKEN_FILE_NAME)
    credentials = storage.get()
    # If no credentials exist, we create new ones
    if credentials is None or credentials.invalid:
        credentials = tools.run_flow(FLOW, storage, flags)
    return credentials
 
def initialize_service():
    # Creates an http object and authorize it using
    # the function prepare_creadentials()
    http = httplib2.Http()
    credentials = prepare_credentials()
    http = credentials.authorize(http)
    # Build the Analytics Service Object with the authorized http object
    return build('analytics', 'v3', http=http)

def get_accounts_ids(service):
    accounts = service.management().accounts().list().execute()
    ids = []
    if accounts.get('items'): # Thsi can be replaced with a cleaner implementation
        for account in accounts['items']:
            ids.append(account['id'])
    return ids
 
if __name__ == '__main__':
    service = initialize_service()
    profiles = service.management().profiles().list(accountId='35423631',
                                                  webPropertyId='~all'
                                                 ).execute()

    print(profiles)
    #accountId='35482361')- this was in admin - drop down
    #webPropertyId='UA-35482361-1').execute()#In property settings detail at the top of the page